# Model Save - SVM (AE Min 8 Node)

### Import List

In [1]:
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import os
import random
import pickle

C:\Users\KSY\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\KSY\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Method Definition

In [2]:
base_dir = "..\\..\\..\\data\\BLE_data\\"
def read_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "raw_data\\in_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "raw_data\\out_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def read_denoised_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "filtered_data\\in_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "filtered_data\\out_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def make_plot(y, num, begin, end):  
    fig = plt.figure(figsize=(15, 5))
    fig.patch.set_facecolor('xkcd:white')
    x = range(len(y))
    plt.bar(x, y, color="blue")
    plt.axis([0, num, begin, end])
    plt.show()

## Start

In [3]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-in data
model_name = 5
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_in_train_noisy, y_in_train_noisy = read_test_data(train_set_num, model_name, "train")
x_in_train_denoised, y_in_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_in_test_noisy, y_in_test_noisy = read_test_data(test_set_num, model_name, "test")
x_in_test_denoised, y_in_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [4]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-out data
model_name = 6
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_out_train_noisy, y_out_train_noisy = read_test_data(train_set_num, model_name, "train")
x_out_train_denoised, y_out_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_out_test_noisy, y_out_test_noisy = read_test_data(test_set_num, model_name, "test")
x_out_test_denoised, y_out_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [5]:
# Car-in
ae_x_in_train_noisy = x_in_train_noisy.astype('float32') / -100.
ae_x_in_train_denoised = x_in_train_denoised.astype('float32') / -100.
ae_x_in_test_noisy = x_in_test_noisy.astype('float32') / -100.
ae_x_in_test_denoised = x_in_test_denoised.astype('float32') / -100.

In [6]:
# Car-out
ae_x_out_train_noisy = x_out_train_noisy.astype('float32') / -100.
ae_x_out_train_denoised = x_out_train_denoised.astype('float32') / -100.
ae_x_out_test_noisy = x_out_test_noisy.astype('float32') / -100.
ae_x_out_test_denoised = x_out_test_denoised.astype('float32') / -100.

## Autoencoder - Denoising (Car in)

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model

autoencoder_01 = load_model('autoencoder_01.h5')

Using TensorFlow backend.


## Autoencoder - Denoising (Car out)

In [8]:
autoencoder_02 = load_model('autoencoder_02.h5')

## Autoencoder - Converting

## Description
- Autoencoder
    - Car in (noisy)를 Autoencoder에 넣은 결과 Car in (denoised)
    - Car in (denoised) 입력
    - Car out (denoised) 정답

In [9]:
autoencoder_03 = load_model('autoencoder_03.h5')

## Autoencoder - Converting

## Description
- Autoencoder
    - Car in (denoised) 입력
    - Car out (denoised) 정답

In [10]:
autoencoder_04 = load_model('autoencoder_04.h5')

## Test with SVM Model

In [11]:
from sklearn.svm import SVC
import numpy as np

def one_hot_convert_normal(data, end_line, output_num):
    list_data = []

    for row in range(0, end_line):
        for column in range(0, output_num):
            if data[row][column] == 1:
                list_data.append(column + 1)
    return list_data

# Description
- SVM_01 (svc_01)
    - Car in (noisy)를 AE_03에 넣으면 결과 Car out (denoised = converted)
    - Car out (denoised = converted)를 입력 (Input)
- SVM_02 (svc_02)
    - Car in (filtered)를 AE_04에 넣어서 결과 Car out (denoised)
    - Car out (denoised)를 입력 (Input)
- SVM_03 (svc_03)
    - Car out (noisy)를 AE_02에 넣으면 Car out (denoised)
    - Car out (denoised)를 입력 (Input)
- SVM_04 (svc_04)
    - Car out (filtered)를 입력 (Input)
- Output (정답)
    - Car out (filtered)의 레이블

In [12]:
# Training data
output_num = 10
end_line = train_set_num * output_num

oh_y_in_train_denoised = one_hot_convert_normal(y_in_train_denoised, end_line, output_num)
oh_y_in_train_denoised = np.array(oh_y_in_train_denoised)

oh_y_out_train_denoised = one_hot_convert_normal(y_out_train_denoised, end_line, output_num)
oh_y_out_train_denoised = np.array(oh_y_out_train_denoised)

# Testing data
end_line = test_set_num * output_num

oh_y_in_test_denoised = one_hot_convert_normal(y_in_test_denoised, end_line, output_num)
oh_y_in_test_denoised = np.array(oh_y_in_test_denoised)

oh_y_out_test_denoised = one_hot_convert_normal(y_out_test_denoised, end_line, output_num)
oh_y_out_test_denoised = np.array(oh_y_out_test_denoised)

## SVM_01

In [13]:
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_in_test_noisy) * (-100)

# SVM training
svc_01 = SVC(C=1000, gamma=100, decision_function_shape='ovo')
svc_01 = svc_01.fit(after_autoencoder_x_train, oh_y_out_train_denoised)

# Save model
pickle.dump(svc_01, open('svc_01.sav', 'wb'))

# SVM accuracy
print("[SVM] Training data accuracy: {:.3f}".format(svc_01.score(after_autoencoder_x_train, oh_y_out_train_denoised)))
print("[SVM] Testing data accuracy: {:.3f}".format(svc_01.score(after_autoencoder_x_test, oh_y_out_test_denoised)))

[SVM] Training data accuracy: 0.763
[SVM] Testing data accuracy: 0.100


## SVM_02

In [14]:
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

# SVM training
svc_02 = SVC(C=1000, gamma=100, decision_function_shape='ovo')
svc_02 = svc_02.fit(after_autoencoder_x_train, oh_y_out_train_denoised)

# Save model
pickle.dump(svc_02, open('svc_02.sav', 'wb'))

# SVM accuracy
print("[SVM] Training data accuracy: {:.3f}".format(svc_02.score(after_autoencoder_x_train, oh_y_out_train_denoised)))
print("[SVM] Testing data accuracy: {:.3f}".format(svc_02.score(after_autoencoder_x_test, oh_y_out_test_denoised)))

[SVM] Training data accuracy: 0.963
[SVM] Testing data accuracy: 0.959


## SVM_03

In [15]:
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

# SVM training
svc_03 = SVC(C=1000, gamma=100, decision_function_shape='ovo')
svc_03 = svc_03.fit(after_autoencoder_x_train, oh_y_out_train_denoised)

# Save model
pickle.dump(svc_03, open('svc_03.sav', 'wb'))

# SVM accuracy
print("[SVM] Training data accuracy: {:.3f}".format(svc_03.score(after_autoencoder_x_train, oh_y_out_train_denoised)))
print("[SVM] Testing data accuracy: {:.3f}".format(svc_03.score(after_autoencoder_x_test, oh_y_out_test_denoised)))

[SVM] Training data accuracy: 0.991
[SVM] Testing data accuracy: 0.976


## SVM_04

In [16]:
# SVM training
svc_04 = SVC(C=1000, gamma=100, decision_function_shape='ovo')
svc_04 = svc_04.fit(x_out_train_denoised, oh_y_out_train_denoised)

# Save model
pickle.dump(svc_04, open('svc_04.sav', 'wb'))

# SVM accuracy
print("[SVM] Training data accuracy: {:.3f}".format(svc_04.score(x_out_train_denoised, oh_y_out_train_denoised)))
print("[SVM] Testing data accuracy: {:.3f}".format(svc_04.score(x_out_test_denoised, oh_y_out_test_denoised)))

[SVM] Training data accuracy: 1.000
[SVM] Testing data accuracy: 0.998
